In [3]:
!pip3 uninstall protobuf -y && pip3 install protobuf==3.20.0

Found existing installation: protobuf 4.21.12
Uninstalling protobuf-4.21.12:
  Successfully uninstalled protobuf-4.21.12
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.7 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.


In [10]:
from math import acos
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from raise_utils.data import DataLoader
from raise_utils.hooks import Hook
import numpy as np
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
file_dic_wang = {"ivy1":     ["ivy-1.4.csv", "ivy-2.0.csv"],
                 "lucene1":  ["lucene-2.0.csv", "lucene-2.2.csv"],
                 "lucene2": ["lucene-2.2.csv", "lucene-2.4.csv"],
                 "poi1":     ["poi-1.5.csv", "poi-2.5.csv"],
                 "poi2": ["poi-2.5.csv", "poi-3.0.csv"],
                 "synapse1": ["synapse-1.0.csv", "synapse-1.1.csv"],
                 "synapse2": ["synapse-1.1.csv", "synapse-1.2.csv"],
                 "camel1": ["camel-1.2.csv", "camel-1.4.csv"],
                 "camel2": ["camel-1.4.csv", "camel-1.6.csv"],
                 "xerces1": ["xerces-1.2.csv", "xerces-1.3.csv"],
                 "jedit1": ["jedit-3.2.csv", "jedit-4.0.csv"],
                 "jedit2": ["jedit-4.0.csv", "jedit-4.1.csv"],
                 "log4j1": ["log4j-1.0.csv", "log4j-1.1.csv"],
                 "xalan1": ["xalan-2.4.csv", "xalan-2.5.csv"]
                 }

In [4]:
def get_data(name: str):
    # For the Wang et al. experiments
    base_path = './DODGE Data/defect/'
    
    def _binarize(x, y): y[y > 1] = 1
    dataset = DataLoader.from_files(
        base_path=base_path, files=file_dic_wang[name], hooks=[Hook('binarize', _binarize)])
    
    return dataset

In [5]:
def fastmap(X, d):
    """
    Perform FastMap dimensionality reduction on the dataset X,
    projecting it onto d dimensions.
    """
    n, m = X.shape
    Y = np.zeros((n, d))
    for i in range(d):
        # Select two points at random
        p, q = np.random.choice(n, 2, replace=False)
        # Compute the distance from each point to the line through p and q
        dists = np.abs(np.cross(X[p] - X[q], X - X[q])) / np.linalg.norm(X[p] - X[q])
        # Normalize the distances
        dists /= np.sum(dists)
        # Use the distances as weights to compute the projection onto the line
        Y[:, i] = dists * (X[p] - X[q]) + X[q]
    return Y

In [6]:
def furthest_point(p, X):
    """
    Find the furthest point from p in X.
    """
    furthest = None
    furthest_distance = 0.
    
    for q in X:
        dist = np.linalg.norm(p - q)
        if dist > furthest_distance:
            furthest_distance = dist
            furthest = q
    
    return furthest, furthest_distance

In [7]:
def get_percent_explained_by_first_component(dataset: str):
    data = get_data(dataset)
    data.x_train = np.array(data.x_train)
    
    pca = PCA(n_components=1)
    pca.fit(data.x_train)
    
    # Use FastMap to find two furthest points
    point_idx = np.random.randint(0, len(data.x_train[0]))
    point = data.x_train[point_idx]
        
    f1, d1 = furthest_point(point, data.x_train)
    f2, d2 = furthest_point(f1, data.x_train)
    
    # Find points along principal diagonal
    p1 = np.array([0] * len(data.x_train[0]))
    p2 = np.array([1] * len(data.x_train[0]))
    
    # Find the direction vectors
    dv1 = (f2 - f1) / np.linalg.norm(f2 - f1)
    dv2 = (p2 - p1) / np.linalg.norm(p2 - p1)
    
    angle = acos(np.dot(dv1, dv2))
    
    return pca.explained_variance_ratio_[0], angle * 180 / 3.14159

In [8]:
for dataset in file_dic_wang:
    explained_var, angle = get_percent_explained_by_first_component(dataset)
    
    print(f'{dataset}: {round(explained_var, 2)} | {angle}')

ivy1: 0.97 | 108.18574983095587
lucene1: 0.92 | 105.34288705705161
lucene2: 0.91 | 105.33535289483842
poi1: 0.85 | 108.33929027774872
poi2: 0.76 | 104.31503993575541
synapse1: 0.69 | 111.45388281794523
synapse2: 0.77 | 112.00828665490202
camel1: 0.87 | 106.41303871510105
camel2: 0.93 | 105.81946479776958
xerces1: 0.96 | 105.83582314130184
jedit1: 0.8 | 103.92822101034967
jedit2: 0.74 | 103.88201299638142
log4j1: 0.92 | 108.08379832522536
xalan1: 0.76 | 109.51023532965712


This means a majority of the variance is explained in one dimension. If this dimension is along the principal diagonal, it would explain why rwfo and wfo perform similarly.

What happens if we do the same but normalize first?

In [13]:
def get_percent_explained_by_first_component_normalized(dataset: str, scaler=Normalizer):
    data = get_data(dataset)
    data.x_train = np.array(data.x_train)
    
    norm = scaler()
    data.x_train = norm.fit_transform(data.x_train)
    
    pca = PCA(n_components=1)
    pca.fit(data.x_train)
    
    # Use FastMap to find two furthest points
    point_idx = np.random.randint(0, len(data.x_train[0]))
    point = data.x_train[point_idx]
        
    f1, d1 = furthest_point(point, data.x_train)
    f2, d2 = furthest_point(f1, data.x_train)
    
    # Find points along principal diagonal
    p1 = np.array([0] * len(data.x_train[0]))
    p2 = np.array([1] * len(data.x_train[0]))
    
    # Find the direction vectors
    dv1 = (f2 - f1) / np.linalg.norm(f2 - f1)
    dv2 = (p2 - p1) / np.linalg.norm(p2 - p1)
    
    angle = acos(np.dot(dv1, dv2))
    
    return pca.explained_variance_ratio_[0], angle * 180 / 3.14159

In [15]:
for dataset in file_dic_wang:
    explained_var, angle = get_percent_explained_by_first_component_normalized(dataset)
    
    print(f'{dataset}: {round(explained_var, 2)} | {angle}')

ivy1: 0.6 | 93.12801599929175
lucene1: 0.6 | 95.2554764535607
lucene2: 0.6 | 95.85701023508356
poi1: 0.5 | 91.20187021788193
poi2: 0.51 | 96.2339134290284
synapse1: 0.66 | 93.93385281276142
synapse2: 0.64 | 87.4965175172689
camel1: 0.48 | 89.95204221923545
camel2: 0.49 | 86.46709528203694
xerces1: 0.54 | 94.47486960950138
jedit1: 0.51 | 93.63378260676038
jedit2: 0.5 | 93.37144467909873
log4j1: 0.61 | 95.75339758123273
xalan1: 0.46 | 97.60283971531152


The first PC is roughly orthogonal to the direction that wfo adds samples. This could mean that part of the reason wfo improves results is that it adds samples along a direction where there aren't many to begin with, so the learner's uncertainty is reduced.